In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [ ]:
!pip install -U albumentations --no-cache-dir
!pip install -U ultralytics wandb
!pip install numpy opencv-python torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.9/914.9 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [ ]:
# Enable W&B logging for Ultralytics

!yolo settings wandb=True

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
JSONDict("/root/.config/Ultralytics/settings.json"):
{
  "settings_version": "0.0.6",
  "datasets_dir": "/content/datasets",
  "weights_dir": "weights",
  "runs_dir": "runs",
  "uuid": "569f3ba64b326db489132663f79cd37279811de477381b83ac131e6cdd129cbb",
  "sync": true,
  "api_key": "",
  "openai_api_key": "",
  "clearml": true,
  "comet": true,
  "dvc": true,
  "hub": true,
  "mlflow": true,
  "neptune": true,
  "raytune": true,
  "tensorboard": true,
  "wandb": true,
  "vscode_msg": true
}
💡 Learn more about Ultralytics Settings at https://docs.ultralytics.com/quickstart/#ultralytics-settings


In [ ]:
# Initialize your Weights & Biases environment

import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: my_object_detection_project (my_object_detection_project-democritus-university-of-thrace) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
# Experiment 1: Freezing the whole network

from ultralytics import YOLO
import wandb
from wandb.integration.ultralytics import add_wandb_callback

wandb.init(project="Transfer_Learning", job_type="training", name="feature_extraction")

model = YOLO('yolov8m.pt')  # Load the YOLOv8 mode

# Add W&B Callback for Ultralytics
add_wandb_callback(model, enable_model_checkpointing=True)

# First Stage (Feature Extraction) --> Freeze the pre-trained convolutional base
# Train the modified detection head
model.train(data='/content/drive/My Drive/Colab Notebooks/Final_Dataset/Dataset.yaml', epochs=30, imgsz=640, batch=16, freeze=22, lr0=0.001, device=0)

wandb.finish()

In [ ]:
# Experiment 2: Freezing the Backbone

from ultralytics import YOLO
import wandb
from wandb.integration.ultralytics import add_wandb_callback

wandb.init(project="Transfer_Learning", job_type="training", name="frozen_backbone")

model = YOLO('yolov8m.pt')  # Load the YOLOv8 mode

# Add W&B Callback for Ultralytics
add_wandb_callback(model, enable_model_checkpointing=True)

# First Stage (Feature Extraction) --> Freeze the pre-trained convolutional base
# Train the modified detection head
model.train(data='/content/drive/My Drive/Colab Notebooks/Final_Dataset/Dataset.yaml', epochs=50, imgsz=640, batch=16, freeze=10, lr0=0.0001, device=0)

wandb.finish()

In [ ]:
# Experiment 3: Freezing the Backbone + tuned hyperparameters

from ultralytics import YOLO
import wandb
from wandb.integration.ultralytics import add_wandb_callback

wandb.init(project="Transfer_Learning", job_type="training", name="frozen_backbone_tuned_hyp_exp3-08")

model = YOLO('yolov8m.pt')  # Load the YOLOv8 mode

tuned_args = {
    "momentum": 0.893668,
    "weight_decay": 0.000165314,
    "warmup_bias_lr": 0.14,
    "warmup_epochs": 4.5,
    "warmup_momentum": 0.738736,
    "box": 0.0853554,
    "cls": 0.436658,
    "dfl": 1.69737,
}

# Add W&B Callback for Ultralytics
add_wandb_callback(model, enable_model_checkpointing=True)

# First Stage (Feature Extraction) --> Freeze the pre-trained convolutional base
# Train the modified detection head
model.train(data='/content/drive/My Drive/Colab Notebooks/Final_Dataset/Dataset.yaml', epochs=50, imgsz=640, batch=16, freeze=10, lr0=0.0001, device=0, **tuned_args)

wandb.finish()

In [ ]:
# Experiment 4: Freezing the Backbone + tuned hyperparameters 2.0

from ultralytics import YOLO
import wandb
from wandb.integration.ultralytics import add_wandb_callback

wandb.init(project="Transfer_Learning", job_type="training", name="frozen_backbone_tuned_hyp_exp4-06")

model = YOLO('yolov8m.pt')  # Load the YOLOv8 mode

tuned_args = {
    "momentum": 0.864377,
    "weight_decay": 2.49441e-05,
    "warmup_bias_lr": 0.11,
    "warmup_epochs": 3.0,
    "warmup_momentum": 0.532624,
    "box": 0.0773716,
    "cls": 0.35085,
    "dfl": 0.773808,
}

# Add W&B Callback for Ultralytics
add_wandb_callback(model, enable_model_checkpointing=True)

# First Stage (Feature Extraction) --> Freeze the pre-trained convolutional base
# Train the modified detection head
model.train(data='/content/drive/My Drive/Colab Notebooks/Final_Dataset/Dataset.yaml', epochs=50, imgsz=640, batch=16, freeze=10, lr0=0.0001, device=0, **tuned_args)

wandb.finish()

In [ ]:
# Experiment 5

from ultralytics import YOLO
import wandb
from wandb.integration.ultralytics import add_wandb_callback

wandb.init(project="Transfer_Learning", job_type="training", name="frozen_backbone_params")

model = YOLO('yolov8m.pt')  # Load the YOLOv8 mode

tuned_args = {
    "momentum": 0.9,
    "weight_decay": 3e-05,
    "warmup_bias_lr": 0.1,
    "warmup_epochs": 5.0,
    "warmup_momentum": 0.75,
    "box": 0.08,
    "cls": 0.4,
    "dfl": 1.0,
}

# Add W&B Callback for Ultralytics
add_wandb_callback(model, enable_model_checkpointing=True)

# First Stage (Feature Extraction) --> Freeze the pre-trained convolutional base
# Train the modified detection head
model.train(data='/content/drive/My Drive/Colab Notebooks/Final_Dataset/Dataset.yaml', seed=42, epochs=100, cache='disk', patience=10, imgsz=640, batch=16, freeze=10, lr0=0.0001, lrf=0.01, device=0, cos_lr=True, **tuned_args)

wandb.finish()

In [ ]:
# Experiment 6

from ultralytics import YOLO
import wandb
from wandb.integration.ultralytics import add_wandb_callback

wandb.init(project="Transfer_Learning", job_type="training", name="frozen_backbone_params02")

model = YOLO('yolov8m.pt')  # Load the YOLOv8 mode


tuned_args = {
    "momentum": 0.893668,
    "weight_decay": 0.000165314,
    "warmup_bias_lr": 0.14,
    "warmup_epochs": 4.5,
    "warmup_momentum": 0.738736,
    "box": 0.0853554,
    "cls": 0.436658,
    "dfl": 1.69737,
}

# Add W&B Callback for Ultralytics
add_wandb_callback(model, enable_model_checkpointing=True)

# First Stage (Feature Extraction) --> Freeze the pre-trained convolutional base
# Train the modified detection head

model.train(
    data='/content/drive/My Drive/Colab Notebooks/Final_Dataset/Dataset.yaml',
    seed=42,
    freeze=10,
    cache='disk',
    epochs=100,
    patience=10,
    imgsz=640,
    batch=16,
    optimizer='AdamW',
    lr0=0.0003,
    lrf=0.1,
    cos_lr=True,
    multi_scale=True,
    device=0,
    **tuned_args,
    mosaic=0.0,
    hsv_h=0.0,
    hsv_s=0.0,
    hsv_v=0.0,
    translate=0.0,
    scale=0.0,
    fliplr=0.0,
    auto_augment=None,
    erasing=0.0,
    crop_fraction=0.0
)

wandb.finish()

In [ ]:
# EXP7

from ultralytics import YOLO
import wandb
from wandb.integration.ultralytics import add_wandb_callback

wandb.init(project="Transfer_Learning", job_type="training", name="frozen_backbone_params03")

model = YOLO('yolov8m.pt')  # Load the YOLOv8 mode


tuned_args = {
    "momentum": 0.864377,
    "weight_decay": 2.49441e-05,
    "warmup_bias_lr": 0.11,
    "warmup_epochs": 3.0,
    "warmup_momentum": 0.532624,
    "box": 0.0773716,
    "cls": 0.35085,
    "dfl": 0.773808,
}

# Add W&B Callback for Ultralytics
add_wandb_callback(model, enable_model_checkpointing=True)

# First Stage (Feature Extraction) --> Freeze the pre-trained convolutional base
# Train the modified detection head

model.train(
    data='/content/drive/My Drive/Colab Notebooks/Final_Dataset/Dataset.yaml',
    seed=42,
    freeze=10,
    cache='disk',
    epochs=100,
    patience=10,
    imgsz=640,
    batch=16,
    optimizer='AdamW',
    lr0=0.0003,
    lrf=0.1,
    cos_lr=True,
    multi_scale=True,
    device=0,
    **tuned_args,
    mosaic=0.0,
    hsv_h=0.0,
    hsv_s=0.0,
    hsv_v=0.0,
    translate=0.0,
    scale=0.0,
    fliplr=0.0,
    auto_augment=None,
    erasing=0.0,
    crop_fraction=0.0
)

wandb.finish()

In [ ]:
# EXP8

from ultralytics import YOLO
import wandb
from wandb.integration.ultralytics import add_wandb_callback
#from ultralytics.data.augment import Albumentations
#from ultralytics.utils import LOGGER, colorstr

wandb.init(project="Transfer_Learning", job_type="training", name="frozen_backbone_params04")

model = YOLO('yolov8m.pt')  # Load the YOLOv8 mode

tuned_args = {
    "momentum": 0.9,
    "weight_decay": 3e-05,
    "warmup_bias_lr": 0.1,
    "warmup_epochs": 5.0,
    "warmup_momentum": 0.75,
    "box": 0.08,
    "cls": 0.4,
    "dfl": 1.0,
}

# Add W&B Callback for Ultralytics
add_wandb_callback(model, enable_model_checkpointing=True)

# First Stage (Feature Extraction) --> Freeze the pre-trained convolutional base
# Train the modified detection head

model.train(
    data='/content/drive/My Drive/Colab Notebooks/Final_Dataset/Dataset.yaml',
    seed=42,
    freeze=10,
    cache='disk',
    epochs=120,
    patience=15,
    imgsz=640,
    batch=16,
    optimizer='AdamW',
    lr0=0.0002,
    lrf=0.005,
    cos_lr=True,
    multi_scale=True,
    device=0,
    **tuned_args,
    mosaic=0.0,
    hsv_h=0.0,
    hsv_s=0.0,
    hsv_v=0.0,
    translate=0.0,
    scale=0.0,
    fliplr=0.0,
    auto_augment=None,
    erasing=0.0,
    crop_fraction=0.0
)

wandb.finish()

In [ ]:
# EXP 9

from ultralytics.data.augment import Albumentations
from ultralytics.utils import LOGGER, colorstr
from ultralytics import YOLO
import torch
import wandb
import cv2
from wandb.integration.ultralytics import add_wandb_callback

wandb.init(project="Transfer_Learning", job_type="training", name="frozen_backbone_augment")

'''
When augmentations are applied dynamically during training, each image can be augmented in different ways each time it is fed to the model.
This approach ensures that the model sees both original and various augmented versions of each image.
'''

emergency_vehicle_labels = [0, 1, 4, 6, 8]

#14 Augmentations
def __init__(self, p=1.0):
    """Initialize the transform object for YOLO bbox formatted params."""
    self.p = p
    self.contains_spatial = True
    self.transform = None
    prefix = colorstr("albumentations: ")
    try:
        import albumentations as A
        T = [
            A.ConstrainedCoarseDropout(
                    num_holes_range=(1, 3),
                    hole_height_range=(0.1, 0.3),
                    hole_width_range=(0.1, 0.3),
                    fill="random_uniform",
                    bbox_labels=emergency_vehicle_labels,
                    p=0.25
                ),
            A.ISONoise (color_shift=(0.01, 0.03), intensity=(0.1, 0.3), p=0.65),
            A.MotionBlur(
                    blur_limit=(3, 7),
                    allow_shifted=False,
                    angle_range=(0, 0),
                    direction_range=(0.8, 1.0),
                    p=0.55
                ),
            A.Sharpen(
                    alpha=(0.2, 0.5),
                    lightness=(0.5, 1.0),
                    method='kernel',
                    kernel_size=5,
                    sigma=1.0,
                    p=0.45
                ),
            A.Affine(
                    scale=(0.9, 1.1),
                    translate_percent=(-0.05, 0.05),
                    rotate=(-3, 3),
                    shear={"x": (-3, 3), "y": (-3, 3)},
                    interpolation=1,
                    mask_interpolation=0,
                    fit_output=True,
                    keep_ratio=True,
                    rotate_method="largest_box",
                    balanced_scale=True,
                    p=0.4
                ),
            A.Perspective(
                    scale=(0.05, 0.07),
                    keep_size=True,
                    fit_output=True,
                    interpolation=1,
                    mask_interpolation=0,
                    border_mode=0,
                    fill=0,
                    fill_mask=0,
                    p=0.4
                ),
              A.OpticalDistortion(
                    distort_limit=(-0.05, 0.05),
                    mode='camera',
                    interpolation=1,
                    mask_interpolation=0,
                    keypoint_remapping_method="mask",
                    p=0.35
                ),
            A.HorizontalFlip(p=0.5),
            A.HueSaturationValue(
                    hue_shift_limit=10,
                    sat_shift_limit=30,
                    val_shift_limit=20,
                    p=0.65
                ),
            A.RandomBrightnessContrast(
                    brightness_limit=(-0.18, 0.18),
                    contrast_limit=(-0.20, 0.20),
                    brightness_by_max=True,
                    ensure_safe_range=True,
                    p=0.45
                ),
            A.RandomRain(
                    slant_range=(-5, 5),
                    drop_length=5,
                    drop_width=1,
                    drop_color=(200, 200, 200),
                    blur_value=5,
                    brightness_coefficient=0.9,
                    rain_type="drizzle",
                    p=0.15
                ),
            A.RandomGamma(
                    gamma_limit=(80, 120),
                    p=0.4
                ),
            A.LongestMaxSize(
                    max_size=640,
                    interpolation=1,
                    p=1.0
                ),
            A.PadIfNeeded(
                    min_height=640,
                    min_width=640,
                    border_mode=0,
                    fill=0,
                    fill_mask=0,
                    p=1.0
                ),
        ]
        self.transform = A.Compose(
            T,
            bbox_params=A.BboxParams(format="yolo", label_fields=["class_labels"], min_area=0, min_visibility=0.0)
        )

        LOGGER.info(prefix + ", ".join(f"{x}".replace("always_apply=False, ", "") for x in T if x.p))
    except ImportError:
        pass
    except Exception as e:
        LOGGER.info(f"{prefix}{e}")

Albumentations.__init__ = __init__

# Load the model
model = YOLO("yolov8m.pt")

# Add W&B Callback for Ultralytics
add_wandb_callback(model, enable_model_checkpointing=True)

'''
I will not be considering the hyperparameters related to the learning rate for final extended train run. This is because, I run each experiment for just 30 epochs, whereas,
for the training of the final model, I want to train longer (100+ epochs) with comparatively lower learning rates. Putting learning rate hyperparameters in the tuning process,
will favor a higher learning rate in the comparatively shorter duration of the runs; however, it might be detrimental when I want to train the model longer.
'''

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tuned_args = {
    "momentum": 0.9,
    "weight_decay": 3e-05,
    "warmup_bias_lr": 0.1,
    "warmup_epochs": 5.0,
    "warmup_momentum": 0.75,
    "box": 0.08,
    "cls": 0.4,
    "dfl": 1.0,
}

# Add W&B Callback for Ultralytics
add_wandb_callback(model, enable_model_checkpointing=True)

wandb.finish()
# Freeze backbone
model.train(data='/content/drive/My Drive/Colab Notebooks/Final_Dataset/Dataset.yaml', device=DEVICE, optimizer='SGD', freeze=10, lr0=0.01, lrf=0.01, seed=42, epochs=120, batch=16,
            imgsz=640, multi_scale=True, cache='disk', cos_lr=True, patience=20, workers=0, **tuned_args, mosaic=0.0, hsv_h=0.0, hsv_s=0.0, hsv_v=0.0, translate=0.0, scale=0.0,
            fliplr=0.0, auto_augment=None, erasing=0.0, crop_fraction=0.0, kobj=0.0, overlap_mask=False, mask_ratio=0, pose=0, close_mosaic=0)

# Finalize the W&B Run
wandb.finish()

**Training & Validation**

In [ ]:
# EXP 10 - FINAL

from ultralytics.data.augment import Albumentations
from ultralytics.utils import LOGGER, colorstr
from ultralytics import YOLO
import torch
import wandb
import cv2
from wandb.integration.ultralytics import add_wandb_callback

wandb.init(project="Transfer_Learning", job_type="training", name="frozen_backbone_final")

'''
When augmentations are applied dynamically during training, each image can be augmented in different ways each time it is fed to the model.
This approach ensures that the model sees both original and various augmented versions of each image.
'''

emergency_vehicle_labels = [0, 1, 4, 6, 8]

#14 Augmentations
def __init__(self, p=1.0):
    """Initialize the transform object for YOLO bbox formatted params."""
    self.p = p
    self.contains_spatial = True
    self.transform = None
    prefix = colorstr("albumentations: ")
    try:
        import albumentations as A
        T = [
            A.ConstrainedCoarseDropout(
                    num_holes_range=(1, 2),
                    hole_height_range=(0.1, 0.2),
                    hole_width_range=(0.1, 0.2),
                    fill="random_uniform",
                    bbox_labels=emergency_vehicle_labels,
                    p=0.3
                ),
            A.ISONoise (color_shift=(0.01, 0.03), intensity=(0.1, 0.3), p=0.5),
            A.MotionBlur(
                    blur_limit=(3, 7),
                    allow_shifted=False,
                    angle_range=(0, 0),
                    direction_range=(0.8, 1.0),
                    p=0.4
                ),
            A.Sharpen(
                    alpha=(0.2, 0.5),
                    lightness=(0.5, 1.0),
                    method='kernel',
                    kernel_size=5,
                    sigma=1.0,
                    p=0.55
                ),
            A.Affine(
                    scale=(0.9, 1.1),
                    translate_percent=(-0.05, 0.05),
                    rotate=(-3, 3),
                    shear={"x": (-3, 3), "y": (-3, 3)},
                    interpolation=1,
                    mask_interpolation=0,
                    fit_output=True,
                    keep_ratio=True,
                    rotate_method="largest_box",
                    balanced_scale=True,
                    p=0.5
                ),
            A.Perspective(
                    scale=(0.05, 0.07),
                    keep_size=True,
                    fit_output=True,
                    interpolation=1,
                    mask_interpolation=0,
                    border_mode=0,
                    fill=0,
                    fill_mask=0,
                    p=0.4
                ),
              A.OpticalDistortion(
                    distort_limit=(-0.05, 0.05),
                    mode='camera',
                    interpolation=1,
                    mask_interpolation=0,
                    keypoint_remapping_method="mask",
                    p=0.25
                ),
            A.HorizontalFlip(p=0.5),
            A.HueSaturationValue(
                    hue_shift_limit=10,
                    sat_shift_limit=30,
                    val_shift_limit=20,
                    p=0.6
                ),
            A.RandomBrightnessContrast(
                    brightness_limit=(-0.15, 0.15),
                    contrast_limit=(-0.20, 0.20),
                    brightness_by_max=True,
                    ensure_safe_range=True,
                    p=0.5
                ),
            A.RandomRain(
                    slant_range=(-5, 5),
                    drop_length=5,
                    drop_width=1,
                    drop_color=(200, 200, 200),
                    blur_value=5,
                    brightness_coefficient=0.9,
                    rain_type="drizzle",
                    p=0.15
                ),
            A.RandomGamma(
                    gamma_limit=(80, 120),
                    p=0.4
                ),
            A.LongestMaxSize(
                    max_size=1024,
                    interpolation=1,
                    p=1.0
                ),
            A.PadIfNeeded(
                    min_height=1024,
                    min_width=1024,
                    border_mode=0,
                    fill=0,
                    fill_mask=0,
                    p=1.0
                ),
        ]
        self.transform = A.Compose(
            T,
            bbox_params=A.BboxParams(format="yolo", label_fields=["class_labels"], min_area=0, min_visibility=0.0)
        )

        LOGGER.info(prefix + ", ".join(f"{x}".replace("always_apply=False, ", "") for x in T if x.p))
    except ImportError:
        pass
    except Exception as e:
        LOGGER.info(f"{prefix}{e}")

Albumentations.__init__ = __init__

# Load the model
model = YOLO("yolov8m.pt")

# Add W&B Callback for Ultralytics
add_wandb_callback(model, enable_model_checkpointing=True)

'''
I will not be considering the hyperparameters related to the learning rate for final extended train run. This is because, I run each experiment for just 30 epochs, whereas,
for the training of the final model, I want to train longer (100+ epochs) with comparatively lower learning rates. Putting learning rate hyperparameters in the tuning process,
will favor a higher learning rate in the comparatively shorter duration of the runs; however, it might be detrimental when I want to train the model longer.
'''

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tuned_args = {
    "momentum": 0.86,
    "weight_decay": 2e-04,
    "warmup_bias_lr": 0.12,
    "warmup_epochs": 4.0,
    "warmup_momentum": 0.7,
    "box": 0.08,
    "cls": 0.35,
    "dfl": 0.85,
}

# Freeze backbone
model.train(data='/content/drive/My Drive/Colab Notebooks/Final_Dataset/Dataset.yaml', device=DEVICE, optimizer='AdamW', freeze=10, lr0=0.0002, lrf=0.1, seed=42, epochs=150,
            batch=-1, imgsz=1024, cache='disk', cos_lr=True, patience=20, **tuned_args, mosaic=0.5, hsv_h=0.0, hsv_s=0.0, hsv_v=0.0, translate=0.0, scale=0.0,
            fliplr=0.0, auto_augment=None, erasing=0.0, crop_fraction=0.0, kobj=0.0, overlap_mask=False, mask_ratio=0, pose=0)

# Finalize the W&B Run
wandb.finish()

wandb: WARNING This integration is tested and supported for ultralytics v8.0.238 and below.
wandb: WARNING             Please report any issues to https://github.com/wandb/wandb/issues with the tag `yolov8`.
wandb: Currently logged in as: my_object_detection_project (my_object_detection_project-democritus-university-of-thrace) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/drive/My Drive/Colab Notebooks/Final_Dataset/Dataset.yaml, epochs=150, time=None, patience=20, batch=-1, imgsz=1024, save=True, save_period=-1, cache=disk, device=cuda, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=10, multi_scale=False, overlap_mask=False, mask_ratio=0, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, format=torchscrip

train: Scanning /content/drive/My Drive/Colab Notebooks/Final_Dataset/labels... 9802 images, 0 backgrounds, 0 corrupt: 100%|██████████| 9802/9802 [00:36<00:00, 266.28it/s]


train: New cache created: /content/drive/My Drive/Colab Notebooks/Final_Dataset/labels.cache


train: Caching images (6.3GB Disk): 100%|██████████| 9802/9802 [00:04<00:00, 2352.53it/s]


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 24/24 [00:22<00:00,  1.09it/s]


                   all       2823      19836      0.891      0.711      0.776      0.536
          police force        699        818      0.917      0.754      0.863      0.607
emergency medical services        954        979      0.926      0.851      0.902      0.678
                   car       2586      13899      0.917      0.615      0.739      0.504
            motorcycle         77         78      0.855      0.385      0.492      0.253
       fire department        162        188       0.92      0.729      0.834      0.601
                   bus        100        109      0.894      0.734      0.787      0.548
        rescue vehicle         35         35      0.714      0.714      0.632      0.379
                 truck       1572       3724        0.9      0.618      0.744      0.504
      military vehicle          3          6      0.974          1      0.995      0.746


Generating Visualizations for batch-1/1:   0%|          | 0/118 [00:00<?, ?it/s]

Speed: 0.2ms preprocess, 2.1ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs/detect/train2


lr/pg0,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr/pg1,▃█████████▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▃▃▃▂▂▂▂▂▂▂▁▁
lr/pg2,▁▃▆██████▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▂▂▂▂▂
metrics/mAP50(B),▁▄▅▆▆▇▇▇▇▇▇▇████████████████████████████
metrics/mAP50-95(B),▁▂▃▄▆▇▇▇▇▇▇▇▇▇▇▇▇███████████████████████
metrics/precision(B),▁▃▄▆▆▇▅▄▇▇▇▇▇▇█▆▇▇▇▇██▇▆▇▇▇██▇▇███▇▇▇▇▇▇
metrics/recall(B),▁▃▃▄▅▅▇▆▇▇▇▇▇█▇██▇██████████████████████
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,█▆▆▅▅▄▄▄▄▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁
